**Import all Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, Lambda
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist

**Load Train and Test Data**

In [ ]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train=X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test=X_test.reshape(X_test.shape[0],28,28,1).astype('float32')
print(X_train.shape)
print(X_test.shape)

**Data Visualization**

In [ ]:
X_train.shape

In [ ]:
X_train_ = X_train.reshape(X_train.shape[0], 28, 28)

fig, axis = plt.subplots(3, 3, figsize=(10, 12))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_train_[i], cmap=plt.get_cmap('binary'))
    ax.set(title = "Real Number is {}".format(y_train[i]));

**One-Hot Encoding**

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]
print(num_classes)

**Data Augmentation**

In [ ]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train,y_train,batch_size=64)

**Normalization**

In [ ]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

**Model Definition**

In [ ]:
def model():
    model=Sequential()
    model.add(Lambda(standardize,input_shape=(28,28,1)))
    model.add(Conv2D(64,(3,3),activation="relu"))
    model.add(Conv2D(64,(3,3),activation="relu"))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation="relu"))
    model.add(Conv2D(128,(3,3),activation="relu"))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),activation="relu"))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(512,activation="relu"))
    model.add(Dense(10,activation="softmax"))
    
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    model.fit_generator(generator=batches,steps_per_epoch=batches.n,epochs=3)
    return model

**Model Training**

In [ ]:
model = model()

**Model Evaluation**

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error:%.2f%%" %(100-score[1]*100))
print(score)

**Prediciting the Outputs**

In [ ]:
X_test = pd.read_csv('../input/test.csv')
X_test = X_test.values.reshape(X_test.shape[0],28,28,1)
y_pred = model.predict_classes(X_test, verbose=1)

**Function to Submit Results to Kaggle**

In [ ]:
def write_preds(y_pred, fname):
    pd.DataFrame({"ImageId":list(range(1,len(y_pred)+1)),"Label":y_pred}).to_csv(fname,index=False,header=True)

**Submit Results**

In [ ]:
 write_preds(y_pred, "keras-cnn-test.csv")